In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from movies import *

In [2]:
pd.plotting.register_matplotlib_converters()
plt.rcParams['figure.figsize'] = [12, 8]
# np.warnings.simplefilter(action='ignore')
np.set_printoptions(linewidth=120, suppress=True)